## Car Prices Prediction - Linear Regression

**=====================================================**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score

import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
from copy import deepcopy
pd.set_option('display.max_columns', None)

### Data Preparation

#### Load Data

In [2]:
X_train = pd.read_csv('/Users/thienla/Desktop/Portfolio/Machine Learning/Module 4/Lab 4/Car_Prices_Poland_train(1).csv')
X_test  = pd.read_csv('/Users/thienla/Desktop/Portfolio/Machine Learning/Module 4/Lab 4/Car_Prices_Poland_test(1).csv')

In [24]:
X_train.shape

(88445, 50)

In [25]:
X_test.shape

(29482, 50)

In [3]:
X_train.head()

,mark,model,generation_name,year,mileage,vol_engine,fuel,city,province,price
0,opel,insignia,gen-a-2008-2017,2009,106125,1796,Gasoline,Częstochowa,Śląskie,25000
1,skoda,fabia,gen-ii-2007,2010,207000,1200,Gasoline,Bydgoszcz,Kujawsko-pomorskie,11900
2,opel,insignia,gen-a-2008-2017,2013,195000,1956,Diesel,Ruda Śląska,Śląskie,24598
3,audi,a4,gen-b8-2007-2015,2012,280000,1968,Diesel,Białystok,Podlaskie,41900
4,citroen,c4-picasso,gen-ii-2013-c4-picasso,2014,122200,1560,Diesel,Dziemionna,Kujawsko-pomorskie,39500


#### Fill Missing Values

In [4]:
# Fill Nulls and NAs
values_to_fill = {}
for col in X_train.drop(columns=['price']).columns:
    if pd.api.types.is_numeric_dtype(X_train[col].dtype):
        values_to_fill[col] = 0
    else:
        values_to_fill[col] = "Missing"

In [5]:
X_train.fillna(value=values_to_fill,inplace=True)
X_test.fillna(value=values_to_fill, inplace=True)

#### Variables Encoding

In [6]:
label_col = 'price'

ohe_orig_columns = ["mark", "fuel"]
te_orig_columns = ["model","generation_name", "city", "province"]
num_orig_columns = ["year","mileage", "vol_engine"]

#### One-Hot-Encoding

One-Hot-Encoding will produce one new column for every valid value of the feature.  

Each column will be populated with 1 if the corresponding valid value found and 0 otherwise. 

In [7]:
all_ohe_columns = []

#Categorical encoders dictionary
cat_encoders = {}

for col in ohe_orig_columns:
    print("One-hot encoding of ", col)
    enc = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    enc.fit(X_train[[col]])
    result = enc.transform(X_train[[col]])
    ohe_columns = [col+"_"+str(x) for x in enc.categories_[0]] 
    result_train = pd.DataFrame(result, columns=ohe_columns)
    X_train = pd.concat([X_train, result_train], axis=1)

# Encode testing
    result = enc.transform(X_test[[col]])
    result_test = pd.DataFrame(result, columns=ohe_columns)
    X_test = pd.concat([X_test, result_test], axis=1)
    all_ohe_columns = all_ohe_columns + ohe_columns

One-hot encoding of  mark
One-hot encoding of  fuel


In [8]:
print("New OHE columns:", all_ohe_columns)

New OHE columns: ['mark_alfa-romeo', 'mark_audi', 'mark_bmw', 'mark_chevrolet', 'mark_citroen', 'mark_fiat', 'mark_ford', 'mark_honda', 'mark_hyundai', 'mark_kia', 'mark_mazda', 'mark_mercedes-benz', 'mark_mini', 'mark_mitsubishi', 'mark_nissan', 'mark_opel', 'mark_peugeot', 'mark_renault', 'mark_seat', 'mark_skoda', 'mark_toyota', 'mark_volkswagen', 'mark_volvo', 'fuel_CNG', 'fuel_Diesel', 'fuel_Electric', 'fuel_Gasoline', 'fuel_Hybrid', 'fuel_LPG']


#### Taget Encoding

In [9]:
te_columns = []

target_encoder = ce.TargetEncoder(cols=te_orig_columns, min_samples_leaf=20, smoothing=10)
target_encoder.fit(X_train[te_orig_columns],X_train[label_col])
X_train_target_enc = target_encoder.transform(X_train[te_orig_columns])
X_test_target_enc = target_encoder.transform(X_test[te_orig_columns])

te_columns = []
for col in te_orig_columns:
    X_train[col+"_te"] = X_train_target_enc[col]
    X_test[col+"_te"] = X_test_target_enc[col]
    te_columns.append(col+"_te")

In [10]:
X_train.head()

,mark,model,generation_name,year,mileage,vol_engine,fuel,city,province,price,mark_alfa-romeo,mark_audi,mark_bmw,mark_chevrolet,mark_citroen,mark_fiat,mark_ford,mark_honda,mark_hyundai,mark_kia,mark_mazda,mark_mercedes-benz,mark_mini,mark_mitsubishi,mark_nissan,mark_opel,mark_peugeot,mark_renault,mark_seat,mark_skoda,mark_toyota,mark_volkswagen,mark_volvo,fuel_CNG,fuel_Diesel,fuel_Electric,fuel_Gasoline,fuel_Hybrid,fuel_LPG,model_te,generation_name_te,city_te,province_te
0,opel,insignia,gen-a-2008-2017,2009,106125,1796,Gasoline,Częstochowa,Śląskie,25000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,48549.435603,34078.467113,93623.392461,81208.395382
1,skoda,fabia,gen-ii-2007,2010,207000,1200,Gasoline,Bydgoszcz,Kujawsko-pomorskie,11900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,31821.475709,16643.232639,94334.920086,69342.069703
2,opel,insignia,gen-a-2008-2017,2013,195000,1956,Diesel,Ruda Śląska,Śląskie,24598,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,48549.435603,34078.467113,39264.196396,81208.395382
3,audi,a4,gen-b8-2007-2015,2012,280000,1968,Diesel,Białystok,Podlaskie,41900,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,53586.996332,42651.432258,78616.011442,67555.811944
4,citroen,c4-picasso,gen-ii-2013-c4-picasso,2014,122200,1560,Diesel,Dziemionna,Kujawsko-pomorskie,39500,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,29098.082418,45995.740967,39210.649854,69342.069703


#### Standardize Variables

I will be calculating un-scaled coefficients, therefore I will scale 
all variables that will be input to the model excluding One-Hot-Encoded columns.

In [11]:
columns_to_scale = num_orig_columns + te_columns
print("Columns to scale: ", columns_to_scale)

Columns to scale:  ['year', 'mileage', 'vol_engine', 'model_te', 'generation_name_te', 'city_te', 'province_te']


In [12]:
num_scalers = {}
cols_numerical_sc = []

# Scale only original numerical columns
for col in X_train[columns_to_scale]:
    print("StandardScaler scale of ", col)
    scaler = StandardScaler()
    scaler.fit(X_train[[col]])
    X_train[col+"_sc"] = scaler.transform(X_train[[col]])
    X_test[col+"_sc"] = scaler.transform(X_test[[col]])

    num_scalers[col] = [deepcopy(scaler),"StandardScaler"]
    cols_numerical_sc.append(col+"_sc")

StandardScaler scale of  year
StandardScaler scale of  mileage
StandardScaler scale of  vol_engine
StandardScaler scale of  model_te
StandardScaler scale of  generation_name_te
StandardScaler scale of  city_te
StandardScaler scale of  province_te


In [13]:
print("Scaled columns:", cols_numerical_sc)

Scaled columns: ['year_sc', 'mileage_sc', 'vol_engine_sc', 'model_te_sc', 'generation_name_te_sc', 'city_te_sc', 'province_te_sc']


### Linear Regression Base Model

To demonstrate multi-colinearity affect, I will first train model without dropping one of the One-Hot-Encoded columns.

In [14]:
train_columns_v0 = cols_numerical_sc + all_ohe_columns
print("Columns to train:", train_columns_v0)

Columns to train: ['year_sc', 'mileage_sc', 'vol_engine_sc', 'model_te_sc', 'generation_name_te_sc', 'city_te_sc', 'province_te_sc', 'mark_alfa-romeo', 'mark_audi', 'mark_bmw', 'mark_chevrolet', 'mark_citroen', 'mark_fiat', 'mark_ford', 'mark_honda', 'mark_hyundai', 'mark_kia', 'mark_mazda', 'mark_mercedes-benz', 'mark_mini', 'mark_mitsubishi', 'mark_nissan', 'mark_opel', 'mark_peugeot', 'mark_renault', 'mark_seat', 'mark_skoda', 'mark_toyota', 'mark_volkswagen', 'mark_volvo', 'fuel_CNG', 'fuel_Diesel', 'fuel_Electric', 'fuel_Gasoline', 'fuel_Hybrid', 'fuel_LPG']


In [15]:
lreg_v0 = LinearRegression()
lreg_v0.fit(X_train[train_columns_v0], X_train.price)

# Calculate R^2 score on Test
print("R^2 on Train:", r2_score(list(X_train.price), 
                                        list(lreg_v0.predict(X_train[train_columns_v0]))))
print("R^2 on Test:", r2_score(list(X_test.price), list(lreg_v0.predict(X_test[train_columns_v0]))))

R^2 on Train: 0.7706977005588165
R^2 on Test: 0.7907060282110733


In [16]:
coef_dict_v0 = dict(zip(train_columns_v0,lreg_v0.coef_))
sorted_tuples = sorted(coef_dict_v0.items(), key=lambda item: item[0])
for key, val in sorted_tuples:
    if "fuel_" in key:
        print(key,":",val)

fuel_CNG : 251430091814356.9
fuel_Diesel : 251430091820271.3
fuel_Electric : 251430091870260.1
fuel_Gasoline : 251430091821189.6
fuel_Hybrid : 251430091847188.2
fuel_LPG : 251430091824901.8


In [17]:
print("Combine Coeficients For Electrical & Fuel:", 
      coef_dict_v0["fuel_Electric"]- coef_dict_v0["fuel_Gasoline"])

Combine Coeficients For Electrical & Fuel: 49070.5


In [18]:
print("Predict Price For First Test Record:",lreg_v0.predict(X_test[train_columns_v0].iloc[[0]]))

Predict Price For First Test Record: [66204.]


### Linear Regression Model After Removing One Of The OHE Columns

I will remove one category column for each of the columns I used OHE to encode:
1. Remove BMW from 'mark' column
2. Remove Gasoline from 'fuel' column

I will set a new "baseline" for the model interpretation: 
- Coeficients for various 'mark_' values will be in relation to BMW
- Coeficients for various 'fuel_' values will be in relation to Gasoline.

In [19]:
train_columns_v1 = cols_numerical_sc + all_ohe_columns
train_columns_v1.remove("mark_bmw")
train_columns_v1.remove("fuel_Gasoline")
print("Columns to train:", train_columns_v1)

Columns to train: ['year_sc', 'mileage_sc', 'vol_engine_sc', 'model_te_sc', 'generation_name_te_sc', 'city_te_sc', 'province_te_sc', 'mark_alfa-romeo', 'mark_audi', 'mark_chevrolet', 'mark_citroen', 'mark_fiat', 'mark_ford', 'mark_honda', 'mark_hyundai', 'mark_kia', 'mark_mazda', 'mark_mercedes-benz', 'mark_mini', 'mark_mitsubishi', 'mark_nissan', 'mark_opel', 'mark_peugeot', 'mark_renault', 'mark_seat', 'mark_skoda', 'mark_toyota', 'mark_volkswagen', 'mark_volvo', 'fuel_CNG', 'fuel_Diesel', 'fuel_Electric', 'fuel_Hybrid', 'fuel_LPG']


In [20]:
lreg_v1 = LinearRegression()
lreg_v1.fit(X_train[train_columns_v1], X_train.price)

# Calculate R^2 score on Test dataset
print("R^2 on Train:", r2_score(list(X_train.price), 
                                        list(lreg_v1.predict(X_train[train_columns_v1]))))
print("R^2 on Test:", r2_score(list(X_test.price), list(lreg_v1.predict(X_test[train_columns_v1]))))

R^2 on Train: 0.7707279670836584
R^2 on Test: 0.7907628135879805


In [21]:
coef_dict_v1 = dict(zip(train_columns_v1,lreg_v1.coef_))
sorted_tuples = sorted(coef_dict_v1.items(), key=lambda item: item[0])
for key, val in sorted_tuples:
    if "fuel_" in key:
        print(key,":",val)

fuel_CNG : 671.6858414589716
fuel_Diesel : -955.4161439631692
fuel_Electric : 49247.28441248662
fuel_Hybrid : 25785.11338309157
fuel_LPG : 2867.0410660811626


In [22]:
print("Predict Price For First Test Record:",lreg_v1.predict(X_test[train_columns_v1].iloc[[0]]))

Predict Price For First Test Record: [66080.47203478]


### Conclusion

After removing one of the OHE columns, it doesn't impact model prediction.

As a result of the removal, I will take care of multi-colinearity which impacts coefficient value as a predictor for the contribution of specific valid value in the categorical variable.

However, the colinearity didn't actually affect the model because number of valid values in the column Fuel was relatively high. As a result, pairwise colinearity was relatively low.